# Analysis on Museum Visitor data

In [6]:
from os import environ as ENV
from psycopg2 import connect
from psycopg2.extensions import connection
from psycopg2.extras import RealDictCursor
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
conn = connect(database=ENV['DB_NAME'], cursor_factory=RealDictCursor)

### 1. What exhibition is most frequently visited?

In [11]:
with conn.cursor() as cur:
    query = """
        SELECT 
            e.public_id Exhibition,
            e.exhibition_name,
            count(exhibition_id) 
        FROM rating_interaction
        join exhibition e using(exhibition_id)
        GROUP BY public_id, e.exhibition_name
        ORDER BY count DESC
        LIMIT 1;
    """
    cur.execute(query)
    most_visited = cur.fetchone()
print(f"Exhibition: {most_visited['exhibition']}, Name: {most_visited['exhibition_name']}, Count: {most_visited['count']}")

Exhibition: EXH_04, Name: Our Polluted World, Count: 481


More people interact with the kiosks if more people visit the exhibits, thus the largest number of ratings coincides with the most frequently visited exhibition.

### What hour of the day has the most ratings?